# 1. Import required libraries and data inspection
*   Import all requierd libraries
*   Import raw data
*   Initial inspection of the data for duplicated rows, missing and data types

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# importing data and transfer it to pandas
drive.mount('/content/drive')
# update path to your CSV location
file_path = '/content/drive/MyDrive/Colab Notebooks/data/raw/retail_store_inventory.csv'
df = pd.read_csv(file_path)
print("***********************************************************************")
print("Show first five rows of the data:")
print("                                 ")
print(df.head())
print("***********************************************************************")
# data inspection for missing / duplicated / data type inconsistency
print(df.info())
print("***********************************************************************")
print(df.describe(include='all'))
print("***********************************************************************")
print(f"number of missing values in each column:")
print(df.isnull().sum())
print("***********************************************************************")
print(f"number of duplicated rows: {df.duplicated().sum()}")
print("***********************************************************************")

Mounted at /content/drive
***********************************************************************
Show first five rows of the data:
                                 
       Date Store ID Product ID     Category Region  Inventory Level  \
0  1/1/2022     S001      P0001    Groceries  North              231   
1  1/1/2022     S001      P0002         Toys  South              204   
2  1/1/2022     S001      P0003         Toys   West              102   
3  1/1/2022     S001      P0004         Toys  North              469   
4  1/1/2022     S001      P0005  Electronics   East              166   

   Units Sold  Units Ordered  Demand Forecast  Price  Discount  \
0         127             55           135.47  33.50        20   
1         150             66           144.04  63.01        20   
2          65             51            74.02  27.99        10   
3          61            164            62.18  32.72        10   
4          14            135             9.26  73.64         0   

  We

According to results of above cell, I conclude:



*   No duplicated rows found in data
*   Following columns have no missing values: 'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast', 'Price', 'Discount'
*   For other columns data type conversion need to be done first


# 2. Data type conversion for rest of columns

In [2]:
df['Date']=pd.to_datetime(df['Date'], errors='coerce')
df['Store ID'] = df['Store ID'].astype("string")
df['Product ID'] = df['Product ID'].astype("string")
df['Category'] = df['Category'].astype("string")
df['Region'] = df['Region'].astype("string")
df['Weather Condition'] = df['Weather Condition'].astype("string")
df['Seasonality'] = df['Seasonality'].astype("string")

# 3. Data inspection after data type conversion

In [3]:
print("***********************************************************************")
# data inspection for missing / duplicated / data type inconsistency
print(df.info())
print("***********************************************************************")
print(df.describe(include='all'))
print("***********************************************************************")
print(f"number of missing values in each column:")
print(df.isnull().sum())
print("***********************************************************************")

***********************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                73100 non-null  datetime64[ns]
 1   Store ID            73100 non-null  string        
 2   Product ID          73100 non-null  string        
 3   Category            73100 non-null  string        
 4   Region              73100 non-null  string        
 5   Inventory Level     73100 non-null  int64         
 6   Units Sold          73100 non-null  int64         
 7   Units Ordered       73100 non-null  int64         
 8   Demand Forecast     73100 non-null  float64       
 9   Price               73100 non-null  float64       
 10  Discount            73100 non-null  int64         
 11  Weather Condition   73100 non-null  string        
 12  Holiday/Promotion   73100 non-

**According to the above results, no missing found in the columns and all data types for each column are same.**

# 4. Encode categorical variables using one-hot encoding.

In [4]:
# Apply one-hot encoding to categorical columns only
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(df[['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Seasonality']])

# Create DataFrame with proper column names
df_onehot = pd.DataFrame(onehot_encoded.toarray(),
                        columns=onehot_encoder.get_feature_names_out(['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Seasonality']))

# Drop original categorical columns and concatenate
df = df.drop(['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Seasonality'], axis=1)
df = pd.concat([df, df_onehot], axis=1)

# convers date to cyclical day of week
df['dayofweek'] = df['Date'].dt.dayofweek
df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)
df = df.drop(['Date', 'dayofweek'], axis=1)


## 5. Save cleaned data into a file

In [5]:
file_path2 = '/content/drive/MyDrive/Colab Notebooks/data/interim/cleaned_data.parquet'
df.to_parquet(file_path2)